In [6]:
import polars as pl
from polars.testing import assert_frame_equal

# Reading the data

In [23]:
transactions = pl.scan_parquet("./Data/100M_transactions.parquet")

In [33]:
%%time
print(transactions.head().collect(engine="gpu"))

shape: (5, 10)
┌────────────┬────────────┬──────────┬─────────────────┬───┬───────┬─────┬───────────────┬────────┐
│ CUST_ID    ┆ START_DATE ┆ END_DATE ┆ TRANS_ID        ┆ … ┆ MONTH ┆ DAY ┆ EXP_TYPE      ┆ AMOUNT │
│ ---        ┆ ---        ┆ ---      ┆ ---             ┆   ┆ ---   ┆ --- ┆ ---           ┆ ---    │
│ str        ┆ date       ┆ date     ┆ str             ┆   ┆ i64   ┆ i64 ┆ str           ┆ f64    │
╞════════════╪════════════╪══════════╪═════════════════╪═══╪═══════╪═════╪═══════════════╪════════╡
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T8I9ZB5A6X90UG8 ┆ … ┆ 9     ┆ 11  ┆ Motor/Travel  ┆ 20.27  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TZ4JSLS7SC7FO9H ┆ … ┆ 2     ┆ 8   ┆ Motor/Travel  ┆ 12.85  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TTUKRDDJ6B6F42H ┆ … ┆ 8     ┆ 1   ┆ Housing       ┆ 383.8  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TDUHFRUKGPPI6HD ┆ … ┆ 3     ┆ 16  ┆ Entertainment ┆ 5.72   │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T0JBZHBMSVRFMMD ┆ … ┆ 5     ┆ 15  ┆ Entertainm

# Which customers have the largest total transctions?

In [32]:
%%time

result = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine="gpu")
)

print(result)

shape: (5, 2)
┌────────────┬──────────┐
│ CUST_ID    ┆ AMOUNT   │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ CP2KXQSX9I ┆ 2.3108e6 │
│ CGOKEO2EH4 ┆ 2.2721e6 │
│ CXYJF3GWQU ┆ 2.2389e6 │
│ C6JC5K02HA ┆ 2.1787e6 │
│ CVH8KQGTUE ┆ 2.1193e6 │
└────────────┴──────────┘
CPU times: user 526 ms, sys: 51.4 ms, total: 578 ms
Wall time: 554 ms


## Which customers have the largest single transaction?

In [29]:
%%time

result = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head(10)
    .collect(engine="gpu")
)

# Pretty print the dataframe
print(result)

shape: (10, 2)
┌────────────┬────────────┐
│ CUST_ID    ┆ max_amount │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ CHF93DNS7X ┆ 6334.35    │
│ CZWNES9XE7 ┆ 6333.97    │
│ C90Z7WVK7W ┆ 6333.4     │
│ CDX7EY0YZZ ┆ 6321.0     │
│ CTLQHZ43M3 ┆ 6317.29    │
│ CNDMIE41TL ┆ 6314.18    │
│ C01218TOCY ┆ 6312.37    │
│ CXDPMWDPIZ ┆ 6303.4     │
│ C9USG5C24Y ┆ 6300.8     │
│ CKD303LHXE ┆ 6300.67    │
└────────────┴────────────┘
CPU times: user 549 ms, sys: 59.4 ms, total: 608 ms
Wall time: 580 ms


## What's the per-month transaction amount for each category over time?

In [31]:
%%time

result = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .head(10)
    .collect(engine="gpu")
)

# Pretty print the dataframe
print(result)

shape: (10, 4)
┌─────────────────────┬──────┬───────┬────────────┐
│ EXP_TYPE            ┆ YEAR ┆ MONTH ┆ AMOUNT     │
│ ---                 ┆ ---  ┆ ---   ┆ ---        │
│ str                 ┆ i64  ┆ i64   ┆ f64        │
╞═════════════════════╪══════╪═══════╪════════════╡
│ Bills and Utilities ┆ 2010 ┆ 1     ┆ 204.85838  │
│ Bills and Utilities ┆ 2010 ┆ 2     ┆ 206.592982 │
│ Bills and Utilities ┆ 2010 ┆ 3     ┆ 212.972266 │
│ Bills and Utilities ┆ 2010 ┆ 4     ┆ 211.853896 │
│ Bills and Utilities ┆ 2010 ┆ 5     ┆ 208.788923 │
│ Bills and Utilities ┆ 2010 ┆ 6     ┆ 210.744436 │
│ Bills and Utilities ┆ 2010 ┆ 7     ┆ 209.58854  │
│ Bills and Utilities ┆ 2010 ┆ 8     ┆ 209.404764 │
│ Bills and Utilities ┆ 2010 ┆ 9     ┆ 210.791806 │
│ Bills and Utilities ┆ 2010 ┆ 10    ┆ 213.101376 │
└─────────────────────┴──────┴───────┴────────────┘
CPU times: user 646 ms, sys: 75.1 ms, total: 721 ms
Wall time: 682 ms
